In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install streamlit -q
!pip install pyngrok
!pip install fastapi uvicorn pydantic requests pyngrok nest-asyncio
!pip install diffusers transformers accelerate scipy safetensors
!pip install torch pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 107.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=c30a649141c109ebb95a0a9b1cf169e0ae1e276a3572577bf9c2e41a533dc301
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token('') # 여기에 ngrok 키가 들어갑니다.

In [ ]:
from fastapi import FastAPI, Response
from pydantic import BaseModel
import uvicorn
import json
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import time

import requests
import numpy as np
import torch
from pydantic import BaseModel
from typing import List
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from torch import autocast


In [ ]:
app = FastAPI()

In [ ]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [ ]:
model_id = "stabilityai/stable-diffusion-2-1"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
class Item(BaseModel):
    image_prompt : str

In [ ]:
import cv2
import numpy as np

def cartoonify(input_path, output_path):
    # 이미지 로드
    image = cv2.imread(input_path)
    if image is None:
        raise ValueError("이미지를 로드할 수 없습니다.")

    # 이미지 사이즈 조정
    scaled_width = 600
    ratio = image.shape[1] / scaled_width
    scaled_height = int(image.shape[0] / ratio)
    image = cv2.resize(image, (scaled_width, scaled_height))

    # 블러 적용하여 노이즈 제거 : 경계선 부드러워짐
    blurred = cv2.medianBlur(image, 5)

    # 그레이스케일 변환
    gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)

    # 경계선 감지 : 두꺼운 경계선
    edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 9)

    # 컬러 이미지로 변환 : 색상이 더 부드러워짐
    color = cv2.bilateralFilter(image, 4, 250, 250)

    # 경계선과 컬러 이미지를 결합하여 카툰 효과 적용
    cartoon = cv2.bitwise_and(color, color, mask=edges)

    # K-means 클러스터링을 통해 색상 감소 : 반비례 / 군집화
    k = 10
    reshaped = cartoon.reshape(-1, 3)
    reshaped = np.float32(reshaped)

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.1)
    _, labels, centers = cv2.kmeans(reshaped, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    centers = np.uint8(centers)
    segmented = centers[labels.flatten()]
    segmented = segmented.reshape(cartoon.shape)

    # 결과 이미지 저장
    cv2.imwrite(output_path, segmented)

# 최적화 코드

# 1개 보내기 및 img to bytes 구현

In [ ]:
global j
j = 0

@app.post("/image")
def get_prompt(item: Item):  # 단일 Item 객체만 받도록 수정
    image_prompt = item.image_prompt
    global j
    with autocast("cuda"):
        image = pipe(image_prompt)[0][0]

    start_byte = 10
    end_byte = 110

    # 바이트 범위에서 잘라낸 후 저장
    image_prompt = image_prompt[start_byte:end_byte]

    img_src = "구글 드라이브 경로"+ image_prompt + str(j) + ".png" # 구글 드라이브 경로가 들어갑니다.
    output_path = "구글 드라이브 경로" + image_prompt + str(j) + ".png" # 구글 드라이브 경로가 들어갑니다.
    image.save(img_src)

    cartoonify(img_src, output_path)

    img = cv2.imread(output_path)

    with open(output_path, "rb") as f:
      img_bytes = f.read()

    j += 1

    response = Response(content=img_bytes, media_type="image/png")
    return response

# 서버 켜기

In [ ]:
import subprocess

# --hostname에 구입한 ngrok 도메인 주소가 들어갑니다.
ngrok_process = subprocess.Popen(["ngrok", "http", "--region=us", "--hostname=", "3000"])

# ngrok 실행에 충분한 시간 대기
import time
time.sleep(3)

# FastAPI 애플리케이션 실행
nest_asyncio.apply()
uvicorn.run(app, host="127.0.0.1", port=3000)


INFO:     Started server process [933]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:3000 (Press CTRL+C to quit)


  0%|          | 0/50 [00:00<?, ?it/s]

 a young man dreaming of becoming the President of the United States in 1997.
요청 처리 시간: 21.243834018707275
INFO:     3.36.58.38:0 - "POST /image HTTP/1.1" 200 OK


  0%|          | 0/50 [00:00<?, ?it/s]

 a man sitting in front of a laptop with a Spanish dictionary open in his lap, a cup of coffee to th
요청 처리 시간: 22.65512490272522
INFO:     3.36.58.38:0 - "POST /image HTTP/1.1" 200 OK


  0%|          | 0/50 [00:00<?, ?it/s]

 a muscular man, with a determined look on his face, lifting a barbell in a gym. He is wearing a whi
요청 처리 시간: 23.05647611618042
INFO:     3.36.58.38:0 - "POST /image HTTP/1.1" 200 OK
